In [1]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import load
from skyfield.positionlib import ICRF, Barycentric
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, EarthLocation
from scipy.interpolate import CubicSpline
import os
import matplotlib.pyplot as plt

# MSE imports
import astro_utils
from astro_utils import jd_to_mjd
from ra_dec import radec2dir, dir2radec, site2geoloc, qv2dir, direction_diff, radec_diff, skyfield_observe
from ra_dec import load_pos_jpl, load_obs_jpl, obs_add_interp_qv, obs_add_calc_dir, obs_add_radec, obs_direction_diff
import asteroid_integrate
from asteroid_data import make_data_one_file, get_earth_pos

### Observation of Earth and Mars according to JPL

In [2]:
# Build DataFrame for earth and mars position
df_earth = load_pos_jpl(body_name='earth', dir_name='../data/jpl//testing/hourly')
df_mars = load_pos_jpl(body_name='mars', dir_name='../data/jpl//testing/hourly')

In [3]:
# Display the earth dataframe
df_earth

,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197.000,2455197.500,1324728,-0.179765,0.970347,-0.000017,-0.017202,-0.003148,8.961125e-07,0.005700,0.986858,0.000038
1,55197.125,2455197.625,1324731,-0.181915,0.969951,-0.000017,-0.017195,-0.003186,8.995828e-07,0.005700,0.986863,0.000039
2,55197.250,2455197.750,1324734,-0.184064,0.969551,-0.000017,-0.017188,-0.003223,9.023349e-07,0.005700,0.986868,0.000039
3,55197.375,2455197.875,1324737,-0.186212,0.969145,-0.000017,-0.017181,-0.003260,9.043645e-07,0.005700,0.986873,0.000040
4,55197.500,2455198.000,1324740,-0.188359,0.968736,-0.000017,-0.017174,-0.003298,9.056684e-07,0.005700,0.986878,0.000041
...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,2458849.000,1412364,-0.161514,0.978014,-0.000019,-0.017273,-0.002832,7.120095e-07,0.005725,0.991261,0.000020
29213,58848.625,2458849.125,1412367,-0.163673,0.977658,-0.000018,-0.017267,-0.002870,6.982338e-07,0.005725,0.991264,0.000020
29214,58848.750,2458849.250,1412370,-0.165831,0.977297,-0.000018,-0.017261,-0.002908,6.842707e-07,0.005725,0.991266,0.000021
29215,58848.875,2458849.375,1412373,-0.167988,0.976931,-0.000018,-0.017254,-0.002946,6.701300e-07,0.005725,0.991269,0.000021


In [4]:
# Display the mars dataframe
# df_mars

In [5]:
# Load the JPL observations of Mars
df_obs_mars_geo = load_obs_jpl(body_name='mars', observer_name='geocenter', dir_name = '../data/jpl/testing/hourly')
df_obs_mars_pal = load_obs_jpl(body_name='mars', observer_name='palomar', dir_name = '../data/jpl/testing/hourly')

# Display the dataframe
df_obs_mars_geo

,mjd,JulianDate,time_key,RA_jpl,DEC_jpl,ux_jpl,uy_jpl,uz_jpl,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,55197.000,2455197.500,1324728,142.327061,18.799029,-0.749289,0.658994,0.065524,142.475968,18.752304,0.738832,-8.970408,6.144676
1,55197.125,2455197.625,1324731,142.309336,18.809991,-0.749061,0.659244,0.065613,142.458277,18.763272,0.738185,-8.940011,6.139300
2,55197.250,2455197.750,1324734,142.291395,18.821012,-0.748831,0.659497,0.065702,142.440369,18.774299,0.737541,-8.909502,6.133941
3,55197.375,2455197.875,1324737,142.273237,18.832093,-0.748598,0.659752,0.065791,142.422245,18.785386,0.736899,-8.878881,6.128601
4,55197.500,2455198.000,1324740,142.254862,18.843232,-0.748362,0.660010,0.065879,142.403904,18.796533,0.736259,-8.848146,6.123279
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,2458849.000,1412364,235.600679,-19.305260,-0.533190,-0.845971,0.006438,235.880046,-19.365556,2.188001,-12.410093,18.197048
29213,58848.625,2458849.125,1412367,235.687152,-19.325324,-0.531949,-0.846752,0.006417,235.966592,-19.385488,2.187105,-12.417570,18.189595
29214,58848.750,2458849.250,1412370,235.773651,-19.345349,-0.530706,-0.847532,0.006395,236.053165,-19.405380,2.186208,-12.425025,18.182137
29215,58848.875,2458849.375,1412373,235.860176,-19.365335,-0.529463,-0.848309,0.006373,236.139763,-19.425233,2.185311,-12.432457,18.174675


In [6]:
# Extract position and velocity of earth from df_earth
q_earth_jpl = np.array([df_earth.X.values, df_earth.Y.values, df_earth.Z.values]) * au
v_earth_jpl = np.array([df_earth.VX.values, df_earth.VY.values, df_earth.VZ.values]) * au / day

# Extract position of mars from df_mars
q_mars_jpl = np.array([df_mars.X.values, df_mars.Y.values, df_mars.Z.values]) * au
v_mars_jpl = np.array([df_mars.VX.values, df_mars.VY.values, df_mars.VZ.values]) * au / day

# Extract obstime_jd, ra, and dec from DataFrame with geocentric observations
obstime_mars_geo_jd = df_obs_mars_geo.JulianDate.values
# ra_mars_geo_jpl = df_obs_mars_geo.RA.values * deg
# dec_mars_geo_jpl = df_obs_mars_geo.DEC.values * deg

# Observation times for palomar observations
obstime_mars_pal_jd = df_obs_mars_pal.JulianDate.values

# Vector of observation times in MJD format
obstime_mars_geo_mjd = jd_to_mjd(obstime_mars_geo_jd)
obstime_mars_pal_mjd = jd_to_mjd(obstime_mars_pal_jd)

# Alias to obstime_mars_mjd because they are the same
obstime_mars_jd = obstime_mars_geo_jd
obstime_mars_mjd = obstime_mars_geo_mjd

### Position & Observation of Earth and Mars according to Skyfield

In [7]:
# Manually load planetary positions using de435
planets_sf = load('../data/jpl/ephemeris/de435.bsp')
earth_sf = planets_sf['earth']
mars_sf = planets_sf['mars barycenter']

# load timescale
ts_sf = load.timescale()

# Generate vector of observation times in Skyfield format
obstime_mars_sf = ts_sf.tt_jd(obstime_mars_jd)

In [8]:
# Observe mars from earth geocenter with Skyfield
ra_mars_geo_sf, dec_mars_geo_sf, delta_mars_geo_sf = \
    skyfield_observe(observer_sf=earth_sf, body_sf=mars_sf, obstime_sf=obstime_mars_sf)

In [9]:
# location of palomar as a Skyfield topos object
geoloc_pal = site2geoloc('palomar', verbose=True)
lon, lat, height = geoloc_pal.geodetic
palomar_topos = skyfield.toposlib.Topos(latitude_degrees=lat.value, longitude_degrees=lon.value, elevation_m=height.value)
palomar_topos

Geolocation of palomar:
cartesian = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geodetic  = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


<Topos 33deg 21' 21.6" N -116deg 51' 46.8" E>

In [10]:
# Observe mars from palomar with Skyfield
palomar_sf = earth_sf + palomar_topos
ra_mars_pal_sf, dec_mars_pal_sf, delta_mars_pal_sf = \
    skyfield_observe(observer_sf=palomar_sf, body_sf=mars_sf, obstime_sf=obstime_mars_sf)

In [11]:
# Load planetary positions and velocities by querying the Skyfield JPL ephemeris interface
# Create them as arrays with bundled astropy units of au and km / second

# Earth
q_earth_sf = earth_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_earth_sf = earth_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

# Palomar
q_palomar_sf = palomar_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_palomar_sf = palomar_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

# Mars
q_mars_sf = mars_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_mars_sf = mars_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

In [12]:
# Demonstrate that q_earth_sf is the same as q_earth_jpl
q_earth_eps = np.mean(np.linalg.norm(q_earth_sf - q_earth_jpl, axis=0))
v_earth_eps = np.mean(np.linalg.norm(v_earth_sf - v_earth_jpl, axis=0))
q_mars_eps = np.mean(np.linalg.norm(q_mars_sf - q_mars_jpl, axis=0))
v_mars_eps = np.mean(np.linalg.norm(v_mars_sf - v_mars_jpl, axis=0))

# Report
print('Difference between Skyfield (JPL ephem) and Horizons download:')
print(f'q_earth : {q_earth_eps:5.3e} au')
print(f'v_earth : {v_earth_eps:5.3e} au / day')
print(f'q_mars  : {q_mars_eps:5.3e} au')
print(f'v_mars  : {v_mars_eps:5.3e} au / day')

Difference between Skyfield (JPL ephem) and Horizons download:
q_earth : 1.406e-09 au
v_earth : 1.626e-08 au / day
q_mars  : 1.565e-09 au
v_mars  : 3.758e-08 au / day


**Conclusion**<br>
Skyfield is essentially identical to JPL in coordinates of Earth and Mars.
Difference is on the order of $10^{-9}$ AU.<br>

### Compare Skyfield vs JPL on RA/DEC of Mars

In [13]:
# Convert SkyField RA/DEC to directions
# u_mars_geo_sf = radec2dir(ra=ra_mars_geo_sf, dec=dec_mars_geo_sf, obstime_mjd=obstime_mars_mjd)
# u_mars_pal_sf = radec2dir(ra=ra_mars_pal_sf, dec=dec_mars_pal_sf, obstime_mjd=obstime_mars_mjd)

In [14]:
# Add RA/DEC and direction from Skyfield to mars observation frames
obs_add_radec(df_obs=df_obs_mars_geo, ra=ra_mars_geo_sf, dec=dec_mars_geo_sf, source='sf')
obs_add_radec(df_obs=df_obs_mars_pal, ra=ra_mars_pal_sf, dec=dec_mars_pal_sf, source='sf')

In [15]:
# Report difference between JPL and Skyfield from geocenter
print(f'Comparing direction of Mars from Geocenter: Skyfield vs. JPL')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC (from observe)\n')
diff_geo_jpl_sf = obs_direction_diff(df_obs=df_obs_mars_geo, src1='jpl', src2='sf', verbose=True)

Comparing direction of Mars from Geocenter: Skyfield vs. JPL
(1) Direction according to JPL: radec2dir applied to JPL RA/DEC
(2) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC (from observe)

Angle Difference: sf vs. jpl
Mean  :   0.000444 deg (   1.598 seconds)
Median:   0.000506 deg (   1.822 seconds)
Max   :   0.000615 deg (   2.213 seconds)


In [16]:
# Report difference between JPL and Skyfield from palomar
print(f'Comparing direction of Mars from Palomar: Skyfield vs. JPL')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC (from observe)\n')
diff_geo_pal_sf = obs_direction_diff(df_obs=df_obs_mars_pal, src1='jpl', src2='sf', verbose=True)

Comparing direction of Mars from Palomar: Skyfield vs. JPL
(1) Direction according to JPL: radec2dir applied to JPL RA/DEC
(2) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC (from observe)

Angle Difference: sf vs. jpl
Mean  :   0.000444 deg (   1.599 seconds)
Median:   0.000506 deg (   1.820 seconds)
Max   :   0.000619 deg (   2.228 seconds)


In [17]:
# Sanity check: difference between palomar and geocenter
radec_diff(name1='geocenter-JPL', name2='palomar-JPL', 
           ra1=df_obs_mars_geo.RA_jpl.values*deg, dec1=df_obs_mars_geo.DEC_jpl.values*deg,
           ra2=df_obs_mars_pal.RA_jpl.values*deg, dec2=df_obs_mars_pal.DEC_jpl.values*deg,
           obstime_mjd=df_obs_mars_geo.mjd.values, verbose=True)

Angle Difference: palomar-JPL vs. geocenter-JPL
Mean  :   0.001409 deg (   5.073 seconds)
Median:   0.001050 deg (   3.780 seconds)
Max   :   0.006340 deg (  22.824 seconds)


In [18]:
# Sanity check: difference between palomar and geocenter
radec_diff(name1='geocenter-Skyfield', name2='palomar-Skyfield', 
           ra1=df_obs_mars_geo.RA_sf.values*deg, dec1=df_obs_mars_geo.DEC_sf.values*deg,
           ra2=df_obs_mars_pal.RA_sf.values*deg, dec2=df_obs_mars_pal.DEC_sf.values*deg,
           obstime_mjd=df_obs_mars_geo.mjd.values, verbose=True)

Angle Difference: palomar-Skyfield vs. geocenter-Skyfield
Mean  :   0.001409 deg (   5.074 seconds)
Median:   0.001050 deg (   3.780 seconds)
Max   :   0.006340 deg (  22.825 seconds)


In [19]:
# Compute direction from Earth to Mars using the RA/DEC from JPL
# u_mars_geo_jpl = radec2dir(ra_mars_geo_jpl, dec_mars_geo_jpl, obstime_mars_geo_mjd)

# Compute direction from Earth to Mars using the RA/DEC from Skyfield
# u_mars_geo_sf = radec2dir(ra_mars_geo_sf, dec_mars_geo_sf, obstime_mars_geo_mjd)

# Report difference
# u_diff_mean = direction_diff(name1='JPL', name2='Skyfield', u1=u_mars_jpl, u2=u_mars_sf, verbose=True)

In [20]:
df_obs_mars_geo

,mjd,JulianDate,time_key,RA_jpl,DEC_jpl,ux_jpl,uy_jpl,uz_jpl,RA_apparent,DEC_apparent,delta,delta_dot,light_time,RA_sf,DEC_sf,ux_sf,uy_sf,uz_sf
0,55197.000,2455197.500,1324728,142.327061,18.799029,-0.749289,0.658994,0.065524,142.475968,18.752304,0.738832,-8.970408,6.144676,142.327169,18.798962,-0.749290,0.658992,0.065524
1,55197.125,2455197.625,1324731,142.309336,18.809991,-0.749061,0.659244,0.065613,142.458277,18.763272,0.738185,-8.940011,6.139300,142.309446,18.809924,-0.749062,0.659242,0.065612
2,55197.250,2455197.750,1324734,142.291395,18.821012,-0.748831,0.659497,0.065702,142.440369,18.774299,0.737541,-8.909502,6.133941,142.291505,18.820944,-0.748832,0.659495,0.065701
3,55197.375,2455197.875,1324737,142.273237,18.832093,-0.748598,0.659752,0.065791,142.422245,18.785386,0.736899,-8.878881,6.128601,142.273349,18.832025,-0.748599,0.659751,0.065790
4,55197.500,2455198.000,1324740,142.254862,18.843232,-0.748362,0.660010,0.065879,142.403904,18.796533,0.736259,-8.848146,6.123279,142.254975,18.843164,-0.748364,0.660009,0.065879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,2458849.000,1412364,235.600679,-19.305260,-0.533190,-0.845971,0.006438,235.880046,-19.365556,2.188001,-12.410093,18.197048,235.600125,-19.305132,-0.533198,-0.845966,0.006438
29213,58848.625,2458849.125,1412367,235.687152,-19.325324,-0.531949,-0.846752,0.006417,235.966592,-19.385488,2.187105,-12.417570,18.189595,235.686598,-19.325196,-0.531957,-0.846747,0.006417
29214,58848.750,2458849.250,1412370,235.773651,-19.345349,-0.530706,-0.847532,0.006395,236.053165,-19.405380,2.186208,-12.425025,18.182137,235.773097,-19.345221,-0.530714,-0.847527,0.006395
29215,58848.875,2458849.375,1412373,235.860176,-19.365335,-0.529463,-0.848309,0.006373,236.139763,-19.425233,2.185311,-12.432457,18.174675,235.859622,-19.365207,-0.529471,-0.848304,0.006374


**Conclusion**<br>
Skyfield is very close to JPL  on observation of Mars.<br>
Mean difference is **1.60 arc seconds** from both Geocenter and Palomar.<br>
The difference between Geocenter and Palomar is about 3.78 arc seconds on average for Mars.<br>
Results appear to be consistent between Geocenter and Palomar.

### Calculate Direction from Earth to Mars with qv2dir() and JPL Position / Velocity

In [21]:
# Add interpolated JPL Positions to observation DataFrames
obs_add_interp_qv(df_obs=df_obs_mars_geo, df_body=df_mars, df_earth=df_earth, source_name='jpl')
obs_add_interp_qv(df_obs=df_obs_mars_pal, df_body=df_mars, df_earth=df_earth, source_name='jpl')

In [22]:
# Display augmented df_obs_mars
df_obs_mars_geo

,mjd,JulianDate,time_key,RA_jpl,DEC_jpl,ux_jpl,uy_jpl,uz_jpl,RA_apparent,DEC_apparent,...,uz_sf,body_x_jpl,body_y_jpl,body_z_jpl,body_vx_jpl,body_vy_jpl,body_vz_jpl,earth_x_jpl,earth_y_jpl,earth_z_jpl
0,55197.000,2455197.500,1324728,142.327061,18.799029,-0.749289,0.658994,0.065524,142.475968,18.752304,...,0.065524,-0.733418,1.457212,0.048394,-0.011980,-0.005093,0.000188,-0.179765,0.970347,-0.000017
1,55197.125,2455197.625,1324731,142.309336,18.809991,-0.749061,0.659244,0.065613,142.458277,18.763272,...,0.065612,-0.734916,1.456575,0.048418,-0.011974,-0.005105,0.000187,-0.181915,0.969951,-0.000017
2,55197.250,2455197.750,1324734,142.291395,18.821012,-0.748831,0.659497,0.065702,142.440369,18.774299,...,0.065701,-0.736412,1.455936,0.048441,-0.011968,-0.005117,0.000187,-0.184064,0.969551,-0.000017
3,55197.375,2455197.875,1324737,142.273237,18.832093,-0.748598,0.659752,0.065791,142.422245,18.785386,...,0.065790,-0.737908,1.455296,0.048464,-0.011961,-0.005130,0.000186,-0.186212,0.969145,-0.000017
4,55197.500,2455198.000,1324740,142.254862,18.843232,-0.748362,0.660010,0.065879,142.403904,18.796533,...,0.065879,-0.739402,1.454654,0.048488,-0.011955,-0.005142,0.000186,-0.188359,0.968736,-0.000017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,2458849.000,1412364,235.600679,-19.305260,-0.533190,-0.845971,0.006438,235.880046,-19.365556,...,0.006438,-1.328050,-0.873097,0.014063,0.008264,-0.010457,-0.000422,-0.161514,0.978014,-0.000019
29213,58848.625,2458849.125,1412367,235.687152,-19.325324,-0.531949,-0.846752,0.006417,235.966592,-19.385488,...,0.006417,-1.327017,-0.874404,0.014010,0.008276,-0.010449,-0.000422,-0.163673,0.977658,-0.000018
29214,58848.750,2458849.250,1412370,235.773651,-19.345349,-0.530706,-0.847532,0.006395,236.053165,-19.405380,...,0.006395,-1.325981,-0.875709,0.013958,0.008288,-0.010441,-0.000422,-0.165831,0.977297,-0.000018
29215,58848.875,2458849.375,1412373,235.860176,-19.365335,-0.529463,-0.848309,0.006373,236.139763,-19.425233,...,0.006374,-1.324945,-0.877014,0.013905,0.008300,-0.010433,-0.000422,-0.167988,0.976931,-0.000018


In [23]:
# Build geolocation of theoretical observer at geocenter
geoloc_geo = site2geoloc(site_name='geocenter', verbose=False)

# Build geolocation of observer at Palomar
geoloc_pal = site2geoloc(site_name='palomar', verbose=True)

Geolocation of palomar:
cartesian = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geodetic  = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


In [24]:
# Compute the directions qv2dir() accounting for observer location; save them to the DataFrame of mars observations
obs_add_calc_dir(df_obs=df_obs_mars_geo, site_name='geocenter', source_name='jpl')
obs_add_calc_dir(df_obs=df_obs_mars_pal, site_name='palomar', source_name='jpl')

In [25]:
# Review added columns
df_obs_mars_geo

,mjd,JulianDate,time_key,RA_jpl,DEC_jpl,ux_jpl,uy_jpl,uz_jpl,RA_apparent,DEC_apparent,...,body_z_jpl,body_vx_jpl,body_vy_jpl,body_vz_jpl,earth_x_jpl,earth_y_jpl,earth_z_jpl,ux_calc_jpl,uy_calc_jpl,uz_calc_jpl
0,55197.000,2455197.500,1324728,142.327163,18.798953,-0.749289,0.658994,0.065524,142.475968,18.752304,...,0.048394,-0.011980,-0.005093,0.000188,-0.179765,0.970347,-0.000017,-0.749290,0.658992,0.065523
1,55197.125,2455197.625,1324731,142.309440,18.809914,-0.749061,0.659244,0.065613,142.458277,18.763272,...,0.048418,-0.011974,-0.005105,0.000187,-0.181915,0.969951,-0.000017,-0.749062,0.659242,0.065612
2,55197.250,2455197.750,1324734,142.291500,18.820935,-0.748831,0.659497,0.065702,142.440369,18.774299,...,0.048441,-0.011968,-0.005117,0.000187,-0.184064,0.969551,-0.000017,-0.748832,0.659495,0.065701
3,55197.375,2455197.875,1324737,142.273343,18.832015,-0.748598,0.659752,0.065791,142.422245,18.785386,...,0.048464,-0.011961,-0.005130,0.000186,-0.186212,0.969145,-0.000017,-0.748599,0.659751,0.065790
4,55197.500,2455198.000,1324740,142.254969,18.843154,-0.748362,0.660010,0.065879,142.403904,18.796533,...,0.048488,-0.011955,-0.005142,0.000186,-0.188359,0.968736,-0.000017,-0.748364,0.660009,0.065879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,2458849.000,1412364,235.600124,-19.305126,-0.533190,-0.845971,0.006438,235.880046,-19.365556,...,0.014063,0.008264,-0.010457,-0.000422,-0.161514,0.978014,-0.000019,-0.533198,-0.845966,0.006438
29213,58848.625,2458849.125,1412367,235.686597,-19.325190,-0.531949,-0.846752,0.006417,235.966592,-19.385488,...,0.014010,0.008276,-0.010449,-0.000422,-0.163673,0.977658,-0.000018,-0.531957,-0.846747,0.006417
29214,58848.750,2458849.250,1412370,235.773096,-19.345215,-0.530706,-0.847532,0.006395,236.053165,-19.405380,...,0.013958,0.008288,-0.010441,-0.000422,-0.165831,0.977297,-0.000018,-0.530714,-0.847527,0.006395
29215,58848.875,2458849.375,1412373,235.859621,-19.365201,-0.529463,-0.848309,0.006373,236.139763,-19.425233,...,0.013905,0.008300,-0.010433,-0.000422,-0.167988,0.976931,-0.000018,-0.529471,-0.848304,0.006374


### Direction from Geocenter to Mars: Compare JPL, Skyfield, and qv2dir(JPL position)

In [26]:
# Report difference for Mars from Geocenter between Skyfiled and MSE calculated
print(f'Comparing direction of Mars from Geocenter: Skyfield vs. MSE calc from JPL positions:')
print(f'(1) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_geo_sf_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_geo, src1='sf', src2='calc_jpl', verbose=True)

Comparing direction of Mars from Geocenter: Skyfield vs. MSE calc from JPL positions:
(1) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC
(2) Direction according to MSE: qv2dir applied to JPL positions & velocities

Angle Difference: calc_jpl vs. sf
Mean  :   0.000008 deg (   0.027 seconds)
Median:   0.000008 deg (   0.029 seconds)
Max   :   0.000012 deg (   0.042 seconds)


In [27]:
# Report difference for Mars from Geocenter between JPL and MSE calculated
print(f'Comparing direction of Mars from Geocenter: JPL vs. MSE calc from JPL positions')
print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_geo_jpl_jpl_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_geo, src1='jpl', src2='calc_jpl', verbose=True)

Comparing direction of Mars from Geocenter: JPL vs. MSE calc from JPL positions
(1) Direction according to JPL: radec2dir applied to JPL RA/DEC
(2) Direction according to MSE: qv2dir applied to JPL positions & velocities

Angle Difference: calc_jpl vs. jpl
Mean  :   0.000446 deg (   1.604 seconds)
Median:   0.000508 deg (   1.830 seconds)
Max   :   0.000616 deg (   2.219 seconds)


**Conclusion**<br>
My calculations are almost identical to Skyfield; accurate to **0.027 arc seconds**<br>
Both Skyfield and I are off from JPL by **1.60 arc seconds**<br>

### Direction from Palomar to Mars: Compare JPL, Skyfield, and qv2dir(JPL position)

In [28]:
from typing import Optional
from ra_dec import infer_shape, light_speed

In [29]:
def qv2dir(q_body: np.ndarray, v_body: np.ndarray, q_earth: np.ndarray, 
           obstime_mjd: Optional[np.ndarray] = None, 
           obsgeoloc: Optional[EarthLocation] = None) -> np.ndarray:
    """
    Compute the direction of displacement from earth to a space body as a unit displacement vector
    u = (ux, uy, uz) in the ecliptic plane.
    INPUTS:
        q_body: position of this body in ecliptic coordinate frame; passed with units (default AU)
        v_body: velocity of this body in ecliptic coordinate frame; passed with units (default AU / day)
        q_earth: position of earth in ecliptic coordinate frame; passed with units (default AU)
        obstime_mjd: observation time as a modified julian date; only required if passing obsgeoloc 
        obsgeoloc: geolocation of the observatory as an astropy EarthLocation object
    RETURNS:
        u: An array [ux, uy, uz] on the unit sphere in the ecliptic frame
    EXAMPLE:
        u = qv2dir(q_body=np.array([-0.328365, 1.570624, 0.040733])*au, 
                   v_body=np.array([-0.013177, -0.001673, 0.000288])*au/day,
                   q_earth=np.array([-0.813785, -0.586761, -0.000003])*au,
                   obsgeoloc=[-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter)
    """
    # compute the correction due to the observatory of obstime_mjd and geoloc are passed
    # dq_obs is the displacement from geocenter to the observatory
    if (obstime_mjd is not None and obsgeoloc is not None):
        # the observation times as astropy time objects
        obstime = astropy.time.Time(obstime_mjd, format='mjd')
        # the displacement from the geocenter to the observatory in the ICRS frame
        x, y, z = obsgeoloc.geocentric
        obs_icrs = SkyCoord(x=x, y=y, z=z, obstime=obstime, frame=ICRS, representation_type='cartesian')
        # displacement from geocenter to observatory in the BME frame
        obs_bme = obs_icrs.transform_to(BarycentricMeanEcliptic)
        dq_topos = obs_bme.cartesian.xyz.to(au)   
    else:
        # default is to use geocenter if obstime and geoloc are not passed
        dq_topos = np.zeros((3)) * au

    # infer data shape
    data_axis, space_axis, shape = infer_shape(q_earth)

    # reshape dq_topos to match q_earth
    dq_topos = dq_topos.reshape(shape)

    # position of the observer in space
    q_obs = q_earth + dq_topos

    # displacement from observer on earth to body; in AU
    q_rel = q_body - q_obs

    # distance; in AU
    r = np.linalg.norm(q_rel, axis=space_axis, keepdims=True) * au
    
    # light time in minutes
    light_time = (r / light_speed)
    
    # adjusted relative position, accounting for light time
    dq_lt = light_time * v_body.to(au/minute)     # convert velocity to au / minute b/c time is in minutes
    q_rel_lt = q_rel - dq_lt
    
    # adjusted direction
    r_lt = np.linalg.norm(q_rel_lt, axis=space_axis, keepdims=True) * au
    u = q_rel_lt / r_lt
    return u.value

In [ ]:
# alias inputs
# q_body = 
# v_body
# q_earth
# obstime_mjd
# obsgeoloc = geoloc_palomar

if (obstime_mjd is not None and obsgeoloc is not None):
    # the observation times as astropy time objects
    obstime = astropy.time.Time(obstime_mjd, format='mjd')
    # the displacement from the geocenter to the observatory in the ICRS frame
    x, y, z = obsgeoloc.geocentric
    obs_icrs = SkyCoord(x=x, y=y, z=z, obstime=obstime, frame=ICRS, representation_type='cartesian')
    # displacement from geocenter to observatory in the BME frame
    obs_bme = obs_icrs.transform_to(BarycentricMeanEcliptic)
    dq_topos = obs_bme.cartesian.xyz.to(au)   
else:
    # default is to use geocenter if obstime and geoloc are not passed
    dq_topos = np.zeros((3)) * au

# infer data shape
data_axis, space_axis, shape = infer_shape(q_earth)

# reshape dq_topos to match q_earth
dq_topos = dq_topos.reshape(shape)

# position of the observer in space
q_obs = q_earth + dq_topos

# displacement from observer on earth to body; in AU
q_rel = q_body - q_obs

# distance; in AU
r = np.linalg.norm(q_rel, axis=space_axis, keepdims=True) * au

# light time in minutes
light_time = (r / light_speed)

# adjusted relative position, accounting for light time
dq_lt = light_time * v_body.to(au/minute)     # convert velocity to au / minute b/c time is in minutes
q_rel_lt = q_rel - dq_lt

# adjusted direction
r_lt = np.linalg.norm(q_rel_lt, axis=space_axis, keepdims=True) * au
u = q_rel_lt / r_lt

In [ ]:
obs_add_calc_dir(df_obs=df_obs_mars_pal, site_name='palomar', source_name='jpl')

In [ ]:
# Report difference for Mars from Palomar between Skyfiled and MSE calculated
print(f'Comparing direction of Mars from Palomar: Skyfield vs. MSE calc from JPL positions:')
print(f'(1) Direction according to Skyfield: radec2dir applied to Skyfield RA/DEC')
print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
diff_pal_sf_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_pal, src1='sf', src2='calc_jpl', verbose=True)

In [ ]:
# Report difference for Mars from Geocenter between JPL and MSE calculated
# print(f'Comparing direction of Mars from Geocenter: JPL vs. MSE calc from JPL positions')
# print(f'(1) Direction according to JPL: radec2dir applied to JPL RA/DEC')
# print(f'(2) Direction according to MSE: qv2dir applied to JPL positions & velocities\n')
# diff_pal_jpl_calc_jpl = obs_direction_diff(df_obs=df_obs_mars_pal, src1='jpl', src2='calc_jpl', verbose=True)

### Vectors of First 16 Asteroids from JPL

In [ ]:
def add_cols_ast_vector(df, asteroid_num: int):
    """Add columns to a DataFrame with asteroid vectors; new columns for asteroid number and mjd"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # Add integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'time_key', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'LT', 'RG', 'RR']
    df = df[columns]
    return df

In [ ]:
# List of dataframes; one per asteroid
df_ast_list = []

# Load the JPL position of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_ast_j = pd.read_csv(f'../data/jpl/testing/daily/vectors-asteroid-{j:03d}.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_ast_j = add_cols_ast_vector(df=df_ast_j, asteroid_num=j)
    # Add this to list of frames
    df_ast_list.append(df_ast_j)

# Concatenate dataframes
df_ast = pd.concat(df_ast_list)

In [ ]:
df_ast

### Observations of First 16 Asteroids from JPL

In [ ]:
def add_cols_ast_observation(df, asteroid_num: int):
    """Add columns to a DataFrame with asteroid vectors; new columns for asteroid number, mjd, and direction"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # Add integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'time_key',
               'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',]
    df = df[columns]
    return df

In [ ]:
# List of dataframes; one per asteroid
df_obs_list = []

# Load the JPL observations of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_obs_j = pd.read_fwf(f'../data/jpl/testing/daily/observer-asteroid-{j:03d}-geocenter.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_obs_j = add_cols_ast_observation(df=df_obs_j, asteroid_num=j)
    # Add this to list of frames
    df_obs_list.append(df_obs_j)

# Concatenate dataframes
df_obs = pd.concat(df_obs_list)

In [ ]:
df_obs

### Calculate Asteroid Direction with qv2dir() and JPL Position / Velocity

In [ ]:
# arrays of asteroid position & velocity from JPL
q_ast_jpl = np.array([df_ast.X.values, df_ast.Y.values, df_ast.Z.values]) * au
v_ast_jpl = np.array([df_ast.VX.values, df_ast.VY.values, df_ast.VZ.values]) * au / day

# tile earth position to match shape of q_ast
q_earth_jpl_tile = np.tile(q_earth_jpl, 16)

# arrays of asteroid angles from JPL
ra_jpl = df_obs.RA.values * deg
dec_jpl = df_obs.DEC.values * deg
obstime_mjd = df_obs.mjd.values.astype(np.float64)

# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_jpl = radec2dir(ra=ra_jpl, dec=dec_jpl, obstime_mjd=obstime_mjd)

# calculate direction with JPL data
u_mse = qv2dir(q_body=q_ast_jpl, v_body=v_ast_jpl, q_earth=q_earth_jpl_tile)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

**Conclusion:<br>**
qv2dir() is highly accurate in computing a right ascension and declination from position and velocity in the barycentric ecliptic plane.<br>
Errors are on the order of **0.87 arc seconds**.<br>
Differences with JPL are due to using a linear approximation to the adjustment of the space body's position due to light lag.  The JPL calculation is iteratively solving for the position of the body on its true orbit at the instant photons leaving it hit the earth at print time.  This simplified calculation is applying an adjustment of the form<br>
```
r = norm(q_body - q_earth)
light_time = r / light_speed
dq = v_body * light_time
```

In [ ]:
# Compute RA and DEC from direction computed by radec2dir() on the JPL data
ra_mse, dec_mse = dir2radec(u_jpl, obstime_mjd)

In [ ]:
# Compute difference in angles
diff_mse = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_mse, dec2=dec_mse, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_mse)
diff_median = np.median(diff_mse)
diff_max = np.max(diff_mse)

# Report results
print(f'Mean Angle Difference: JPL vs. MSE from JPL positions')
print(f'Mean  : {diff_mean:5.3e} seconds')
print(f'Median: {diff_median:5.3e} seconds')
print(f'Max   : {diff_max:5.3e} seconds')

**Conclusion:<br>**
The round trip of radec2dir() and dir2radec() is accurate on the order of double precision.<br>
In the test, a direction was computed from the RA and DEC provided by JPL. This was then converted back to a RA and DEC.<br>
Errors are on the order of **5.8E-11 arc seconds**.<br>

### Calculate Asteroid Direction with qv2dir() and MSE Position / Velocity

In [ ]:
ast_elt = asteroid_integrate.load_data()
ast_elt.rename(mapper={'Num':'asteroid_num'}, axis='columns', inplace=True)

In [ ]:
# Range of asteroids to for data
ast_num_file_start: int = 1
ast_num_file_end: int = 1000
inputs, outputs = make_data_one_file(0, ast_num_file_end)

In [ ]:
ast_elt

In [ ]:
# The block of asteroid numbers to test (inclusive boundaries)
ast_num_min = 1
ast_num_max = 16

# The number of asteroids, times, and total rows we want to match
N_ast = ast_num_max - ast_num_min + 1
N_t = df_earth.mjd.size
N_row = N_ast * N_t

# Report data shape
print(f'Shape of data frames df_ast and df_obs:')
print(f'N_ast = {N_ast:5} asteroids')
print(f'N_t   = {N_t:5} observation times')
print(f'N_row = {N_row:5} rows in df_ast and df_obs')

In [ ]:
# Filter for asteroid numbers 
ast_num_file = np.arange(ast_num_file_start, ast_num_file_end, dtype=np.int64)
mask_ast = (ast_num_min <= ast_num_file) & (ast_num_file <= ast_num_max)

# MSE integrated times as one array
ts = inputs['ts'][0]

# Time range for JPL data
t_min = np.min(obstime_mjd)
t_max = np.max(obstime_mjd)

# Filter for MSE times that match
mask_t = (t_min <= ts) & (ts <= t_max)

In [ ]:
# Block of asteroid data 
q_ast_all = outputs['q']
v_ast_all = outputs['v']

# filter for selected asteroids only
q_ast_all_t = q_ast_all[mask_ast, :, :]
v_ast_all_t = v_ast_all[mask_ast, :, :]

# filter for selected times only
q_ast_mse_3d = q_ast_all_t[:, mask_t, :]
v_ast_mse_3d = v_ast_all_t[:, mask_t, :]

# for some reason i don't understand, can't do these at once
# q_ast_mse = q_ast_all[mask_ast, mask_t, :]
q_ast_mse_3d.shape

In [ ]:
# Get position of Earth using utility function
q_earth_mse_tile = get_earth_pos(obstime_mjd).transpose()

q_earth_mse_tile.shape

In [ ]:
# shape JPL positions to match q_ast_mse with three axes (asteroid_num, time_idx, space_dim)
q_ast_jpl_3d = np.zeros((N_ast, N_t, 3))
q_ast_jpl_3d[:, :, 0] = df_ast.X.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 1] = df_ast.Y.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 2] = df_ast.Z.values.reshape((N_ast, N_t))
q_ast_jpl_3d.shape

In [ ]:
# Reshape MSE asteroid data to match shape of DataFrame
q_ast_mse = np.zeros((3, N_row))
v_ast_mse = np.zeros((3, N_row))

In [ ]:
# Position
q_ast_mse[0, :] = q_ast_mse_3d[:, :, 0].reshape((-1))
q_ast_mse[1, :] = q_ast_mse_3d[:, :, 1].reshape((-1))
q_ast_mse[2, :] = q_ast_mse_3d[:, :, 2].reshape((-1))

# Velocity
v_ast_mse[0, :] = v_ast_mse_3d[:, :, 0].reshape((-1))
v_ast_mse[1, :] = v_ast_mse_3d[:, :, 1].reshape((-1))
v_ast_mse[2, :] = v_ast_mse_3d[:, :, 2].reshape((-1))

In [ ]:
# Compute direction from MSE position and velocity
u_mse = qv2dir(q_body=q_ast_mse*au, v_body=v_ast_mse*au/day, q_earth=q_earth_mse_tile*au)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

**Conclusion:<br>**
My end to end calculation of astromentric RA and DEC are very close to those of JPL.<br>
In my calculation, I am only taking a single snapshot of planetary positions and velocities, plus orbital elements of the asteroids.  Everything else is done by numerically integrating the system in rebound.
I am computing an astrometric direction u on the unit sphere in the ecliptic frame, and comparing this to a direction from JPL.  The JPL direction u_jpl is computed by applying radec2dir() on the quoted RA and DEC.<br>
Errors are on the order of **3.6 arc seconds**.<br>
I am guessing that one main source for the difference with JPL is that I used heliocentric rather than barycentric coordinates when saving the outputs of the rebound integration.  I plan to switch to barycentric for the asteroid search.  Of course there are also some other differences because these are completely separate calculations.  JPL in particular is using many more massive bodies, and they are accounting for relativistic effects.<br>
Still, the bottom line is that an agreement of only 3.6 arc seconds is very tight and suggests that my methodology is basically sound.

### Astrometric vs. Apparent Coordinates

In [ ]:
df_obs

**JPL Definitions of Astrometric & Apparent RA/DEC**

 R.A._______(ICRF)_______DEC =
  Astrometric right ascension and declination of the target center with
respect to the observing site (coordinate origin) in the reference frame of
the planetary ephemeris (ICRF). Compensated for down-leg light-time delay
aberration.

  Units: RA  in decimal degrees (ddd.fffffffff)
         DEC in decimal degrees (sdd.fffffffff)

 
 R.A._______(airless-appar)_______DEC. =
  Airless apparent right ascension and declination of the target center with
respect to an instantaneous reference frame defined by the Earth equator
of-date (z-axis) and meridian containing the Earth equinox of-date (x-axis,
IAU76/80). Compensated for down-leg light-time delay, gravitational deflection
of light, stellar aberration, precession & nutation. Note: equinox (RA origin)
is offset -53 mas from the of-date frame defined by the IAU06/00a P & N system.

  Units: RA  in decimal degrees (ddd.fffffffff)
         DEC in decimal degrees (sdd.fffffffff)

In [ ]:
# alias the astrometric RA/DEC so the variable names look consistent
# these are the astrometric RA/DEC
ra_astro = ra_jpl
dec_astro = dec_jpl

# arrays of apparent asteroid angles from JPL
ra_appar = df_obs.RA_apparent.values * deg
dec_appar = df_obs.DEC_apparent.values * deg

# Compute difference in angles
diff_app = radec_diff('Astrometric', 'Apparent', ra1=ra_astro, dec1=dec_astro, ra2=ra_appar, dec2=dec_appar, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_app)
diff_median = np.median(diff_app)
diff_max = np.max(diff_app)

# Report results
print(f'Mean Angle Difference: JPL astrometric vs. JPL apparent')
print(f'Mean  : {diff_mean:5.0f} seconds ({(diff_mean/3600):0.3f} degrees)')
print(f'Median: {diff_median:5.0f} seconds ({(diff_median/3600):0.3f} degrees)')
print(f'Max   : {diff_max:5.0f} seconds ({(diff_max/3600):0.3f} degrees)')

**Conclusion**<br>
The difference between astrometric and apparent RA / DEC is really important!<br>
It's much more important than some of the other effects considered.<br>
It introduces errors on the order of **0.21 degrees / 743 arc seconds**<br>
We need to figure out the quotation basis of the ZTF data!<br>
Francisco from Alerce says he believes ZTF data "must be" astrometric.  Hopefully this is correct!

### Estimate Importance of Including Observatory Location

In [ ]:
# Alias old dataframe to df_geocenter
df_geo_daily = df_obs.copy()

In [ ]:
# Load a single data file with Ceres viewed daily from Palomar
# Times will match against df_geocenter exactly

df_pal_daily = pd.read_csv(f'../data/jpl/testing/daily/observer-asteroid-001-palomar.txt', index_col=False)
df_pal_daily = add_cols_ast_observation(df_pal_daily, asteroid_num=1)

In [ ]:
df_pal_daily

In [ ]:
# only rows in df_geocenter for ceres
mask = df_geo_daily.asteroid_num == 1

# extract RA and DEC for geocenter
ra_geo = df_geo_daily[mask].RA.values * deg
dec_geo = df_geo_daily[mask].DEC.values * deg

# extract RA and DEC for palomar
ra_pal = df_pal_daily.RA.values * deg
dec_pal = df_pal_daily.DEC.values * deg

In [ ]:
# report the angle difference from accounting the observatory
diff_topos = radec_diff('Geocenter', 'Palomar', ra1=ra_geo, dec1=dec_geo, ra2=ra_pal, dec2=dec_pal, 
                         obstime_mjd=obstime_mjd, verbose=True)

**Conclusion**<br>
Ignoring the observatory location would introduce an error of **2.55 arc seconds**<br>
This effect is important enough that we should certainly try to model it.

### Incorporate Observatory Location (topos)

In [ ]:
def add_cols_ast_obs_site(df, asteroid_num: int, filter_daylight: bool = False):
    """Add columns to a DataFrame with observation from a real site on earth; new columns for asteroid number, mjd, and direction"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # Add integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Compute flag indicating whether observation is possible (no direct sunlight; allow twilight and moonlight)
    df['CanObserve'] = (df['SunIsPresent'] != '*')
    # Filter to only rows where observation is possible if requested
    if filter_daylight:
        mask = df['CanObserve'] == True
        df = df[mask]
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'time_key',
               'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',
               # 'SunIsPresent', 'MoonIsPresent',
              ]
    df = df[columns]
    return df

In [ ]:
# Load the JPL position of Earth at 3 hour intervals as CSV
df_earth_3h = pd.read_csv('../data/jpl/testing/hourly/vectors-earth.txt', index_col=False)
# Add columns for date /time
df_earth_3h = add_col_planet(df_earth_3h)

# Display the earth dataframe
df_earth_3h

In [ ]:
def make_vectors_hourly(ast_num0: int, ast_num1: int):
    """Build asteroid vectors DataFrame from hourly observations"""
    # List of dataframes; one per asteroid
    df_ast_list = []

    # Load the JPL position of asteroids one at a time
    for j in range(ast_num0, ast_num1+1):
        # print(j)
        df_ast_j = pd.read_csv(f'../data/jpl/testing/hourly/vectors-asteroid-{j:03d}.txt', index_col=False)
        # Add columns for the asteroid_num and mjd
        df_ast_j = add_cols_ast_vector(df=df_ast_j, asteroid_num=j)
        # Add this to list of frames
        df_ast_list.append(df_ast_j)

    # Concatenate dataframes
    df_ast = pd.concat(df_ast_list)
    
    return df_ast

In [ ]:
# Build asteroid vectors at 3 hour intervals
df_ast_3h = make_vectors_hourly(ast_num0=1, ast_num1=1)

In [ ]:
df_ast_3h

In [ ]:
def make_obs_hourly(ast_num0: int, ast_num1: int, location_name: str):
    """Build observation DataFrame from hourly observations"""
    # List of dataframes; one per asteroid
    df_obs_list = []

    # Load the JPL observations of asteroids one at a time
    for j in range(ast_num0, ast_num1+1):
        # print(j)
        df_obs_j = pd.read_csv(f'../data/jpl/testing/hourly/observer-asteroid-{j:03d}-{location_name}.txt', index_col=False)
        # Add columns for the asteroid_num and mjd
        df_obs_j = add_cols_ast_obs_site(df=df_obs_j, asteroid_num=j, filter_daylight=False)
        # Add this to list of frames
        df_obs_list.append(df_obs_j)

    # Concatenate dataframes
    df_obs = pd.concat(df_obs_list)

    return df_obs

In [ ]:
# Load palomar observations at 3 hour intervals, filtered to eliminate daylight
df_pal_3h = make_obs_hourly(ast_num0=1, ast_num1=1, location_name='palomar')

# Load earth observations at 3 hour intervals
df_geo_3h = make_obs_hourly(ast_num0=1, ast_num1=1, location_name='geocenter')

In [ ]:
df_pal_3h

In [ ]:
df_geo_3h

In [ ]:
# Earth location of the Palomar observatory
geoloc_palomar = EarthLocation.of_site('Palomar')
print(f'geolocation of Palomar observatory:')
print(f'geoloc_palomar = {geoloc_palomar}')
print(f'geoloc_palomar geodetic = {geoloc_palomar.geodetic}')

In [ ]:
# For some reason, the 'nice' way to do this below leads to a bug in constructor to GCRS
# geoloc_palomar = EarthLocation(-2410346.8, -4758666.8, 3487943, unit='m')

# Workaround: extract the geolocation of the Palomar observatory using EarthLocation.of_site
obsgeoloc = np.array([-2410346.78217658, -4758666.82504051, 3487942.97502457]) * meter
obsgeoloc

**Make sure calculations from geocenter still work**

In [ ]:
# extract the position and velocity of the asteroids and earth at 3 hour intervals according to JPL
q_ast_jpl_3h = np.array([df_ast_3h.X.values, df_ast_3h.Y.values, df_ast_3h.Z.values]) * au
v_ast_jpl_3h = np.array([df_ast_3h.VX.values, df_ast_3h.VY.values, df_ast_3h.VZ.values]) * au / day
q_earth_jpl_3h = np.array([df_earth_3h.X.values, df_earth_3h.Y.values, df_earth_3h.Z.values]) * au
num_ast = len(set(df_ast_3h.asteroid_num))
q_earth_jpl_tile_3h = np.tile(q_earth_jpl_3h, num_ast)

# alias angles for JPL observations at geocenter sampled every 3 hours
ra_geo_jpl_3h = df_geo_3h.RA.values * deg
dec_geo_jpl_3h = df_geo_3h.DEC.values * deg
obstime_mjd_3h = df_geo_3h.mjd.values

In [ ]:
# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_geo_jpl_3h = radec2dir(ra=ra_jpl_3h, dec=dec_jpl_3h, obstime_mjd=obstime_mjd_3h)

# calculate direction with JPL data
u_geo_mse_3h = qv2dir(q_body=q_ast_jpl_3h, v_body=v_ast_jpl_3h, q_earth=q_earth_jpl_tile_3h)

# difference in directions as a vector
u_diff = u_geo_mse_3h - u_geo_jpl_3h

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

**Observations from Palomar According to JPL; Augment wit Interpolated Earth & Asteroid Position**

In [ ]:
def obs_add_interp_qv(df_obs: pd.DataFrame, 
                      df_ast: pd.DataFrame,
                      df_earth: pd.DataFrame,) -> None:
    """Add interpolated position and velocity to a DataFrame of asteroid observations"""
    
    # Get list of distinct asteroid numbers
    asteroid_nums = list(set(df_obs.asteroid_num.values))

    # Add new columns for position and velocity
    cols = ['ast_x', 'ast_y', 'ast_z', 'ast_vx', 'ast_vy', 'ast_vz']
    for col in cols:
        df_obs[col] = 0.0
        
    # Interpolator for earth position
    interp_t = df_earth.mjd.values
    interp_q = df_earth[['X', 'Y', 'Z']].values
    interp_earth = CubicSpline(x=interp_t, y=interp_q)
    
    # Set interpolated position of earth
    earth_cols = ['earth_x', 'earth_y', 'earth_z']
    earth_q = interp_earth(df_obs.mjd.values)
    for k, col in enumerate(earth_cols):
        df_obs[col] = earth_q[:, k]

    # Add interpolated position and velocity to df_palomar
    for asteroid_num in asteroid_nums:

        # Masks for this asteroid on the vector and observation dataframes
        mask_vec = (df_ast.asteroid_num == asteroid_num)
        mask_obs = (df_obs.asteroid_num == asteroid_num)

        # Build an interpolator for the asteroid position and velocity
        interp_t = df_ast[mask_vec].mjd.values
        interp_qv = df_ast[mask_vec][['X', 'Y', 'Z', 'VX', 'VY', 'VZ']].values
        interp_ast = CubicSpline(x=interp_t, y=interp_qv)

        # The times to be interpolated
        obs_t = df_obs[mask_obs].mjd.values

        # Evaluate the interpolated position / velocity at the observation times
        obs_qv = interp_ast(obs_t)

        # Assign interpolated qv to the df_obs dataframe on the mask
        for k, col in enumerate(cols):
            df_obs.loc[mask_obs, col] = obs_qv[:, k]

In [ ]:
# Add interpolated position and velocity variables according to JPL
obs_add_interp_qv(df_ast=df_ast_3h, df_earth=df_earth_3h, df_obs=df_pal_3h)

In [ ]:
# Review the augmented observation table
df_pal_3h

In [ ]:
from typing import Optional
light_speed = astropy.constants.c.to(au / minute)

In [ ]:
zero_deg = 0.0 * deg
zero_meter = 0.0 * meter
zero_au = 0.0 * au

# The observation time
obstime = astropy.time.Time(58600.0, format='mjd')

In [ ]:
def qv2dir(q_body: np.ndarray, v_body: np.ndarray, q_earth: np.ndarray, 
           obstime_mjd: Optional[np.ndarray] = None, obsgeoloc: Optional[np.ndarray] = None) -> np.ndarray:
    """
    Compute the direction of displacement from earth to a space body as a unit displacement vector
    u = (ux, uy, uz) in the ecliptic plane.
    INPUTS:
        q_body: position of this body in ecliptic coordinate frame; passed with units (default AU)
        v_body: velocity of this body in ecliptic coordinate frame; passed with units (default AU / day)
        q_earth: position of earth in ecliptic coordinate frame; passed with units (default AU)
        obstime_mjd: observation time as a modified julian date; only required if passing obsgeoloc 
        obsgeoloc: geolocation of the observatory as vector [X, Y, Z] in meters using ITRS
    RETURNS:
        u: An array [ux, uy, uz] on the unit sphere in the ecliptic frame
    EXAMPLE:
        u = qv2dir(q_body=np.array([-0.328365, 1.570624, 0.040733])*au, 
                   v_body=np.array([-0.013177, -0.001673, 0.000288])*au/day,
                   q_earth=np.array([-0.813785, -0.586761, -0.000003])*au,
                   obsgeoloc=[-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter)
    """
    # compute the correction due to the observatory of obstime_mjd and geoloc are passed
    # dq_topos is the displacement from geocenter to the observatory
    if (obstime_mjd is not None and obsgeoloc is not None):
        # the observation times as astropy time objects
        obstime = astropy.time.Time(obstime_mjd, format='mjd')
        # the displacement from the geocenter to the observatory in the ICRS frame
        x, y, z = obsgeoloc
        obs_icrs = SkyCoord(x=x, y=y, z=z, obstime=obstime, frame=ICRS, representation_type='cartesian')
        # displacement from geocenter to observatory in the BME frame
        obs_bme = obs_icrs.transform_to(BarycentricMeanEcliptic)
        dq_topos = obs_bme.cartesian.xyz.to(au).reshape((-1, 1))
    else:
        # default is to use geocenter if obstime and geoloc are note passed
        dq_topos = np.zeros((3,1)) * au
        
    # position of the observer in space
    q_obs = q_earth + dq_topos

    # displacement from observer on earth to body; in AU
    q_rel = q_body - q_obs

    # distance; in AU
    r = np.linalg.norm(q_rel, axis=0) * au
    
    # light time in minutes
    light_time = (r / light_speed)
    
    # adjusted relative position, accounting for light time
    dq_lt = light_time * v_body.to(au/minute)     # convert velocity to au / minute b/c time is in minutes
    q_rel_lt = q_rel - dq_lt
    
    # adjusted direction
    r_lt = np.linalg.norm(q_rel_lt, axis=0) * au
    u = q_rel_lt / r_lt
    return u.value

In [ ]:
# extract the position and velocity according to JPL; same alignment as Palomar obs data
q_ast_pal_jpl = df_pal_3h[['ast_x', 'ast_y', 'ast_z']].values.transpose() * au
v_ast_pal_jpl = df_pal_3h[['ast_vx', 'ast_vy', 'ast_vz']].values.transpose() * au / day
q_earth_pal_jpl = df_pal_3h[['earth_x', 'earth_y', 'earth_z']].values.transpose() * au

# extract the RA and DEC from Palomar according to JPL
ra_pal_jpl = df_pal_3h.RA.values * deg
dec_pal_jpl = df_pal_3h.DEC.values * deg
obstime_pal_mjd = df_pal_3h.mjd.values

In [ ]:
# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_pal_jpl = radec2dir(ra=ra_pal_jpl, dec=dec_pal_jpl, obstime_mjd=obstime_pal_mjd)

# calculate direction with JPL data
u_pal_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl, obstime_mjd=obstime_pal_mjd, obsgeoloc=geoloc_palomar)

# difference in directions as a vector
u_diff = u_pal_mse - u_pal_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

In [ ]:
u_pal_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl, obstime_mjd=obstime_pal_mjd, obsgeoloc=geoloc_palomar)
# u_geo_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl, obstime_mjd=obstime_pal_mjd, obsgeoloc=np.zeros(3)*meter)
u_geo_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl)

In [ ]:
# difference between MSE palomar and MSE geocenter; expect this to be around 3 arc seconds
np.rad2deg(np.mean(np.linalg.norm(u_pal_mse - u_geo_mse, axis=0)))*3600

In [ ]:
# difference between JPL palomar and MSE from geocenter; expect error to be larger by about 3 arc seconds
np.rad2deg(np.mean(np.linalg.norm(u_geo_mse - u_pal_jpl, axis=0)))*3600

In [ ]:
# difference between MSE palomar and JPL palomar; want this to be very small
np.rad2deg(np.mean(np.linalg.norm(u_pal_mse - u_pal_jpl, axis=0)))*3600